# Generate Validation .npy files

In [ ]:
from src.scaled.dataset.particle_fluid_dataset import ParticleFluidDataset
from torch.utils.data import DataLoader
import numpy as np

val_dataset = ParticleFluidDataset(
        data_dir="data/couple_spout_3D",
        skip_timestep=1,
        time_steps_list=[i for i in range(200, 250)],
    )

val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=1)

for i, data in enumerate(val_dataloader):
    current_data = data[0][0]

    current_data_np = current_data.numpy()
    np.save(f"data/evaluation_npy_step200-250/gt_{i:03d}.npy", current_data_np)

# Generate Whole Rollout .npy files

In [ ]:
val_dataset = ParticleFluidDataset(
        data_dir="data/couple_spout_3D",
        skip_timestep=1,
        time_steps_list=[i for i in range(1, 250)],
    )

val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=1)

for i, data in enumerate(val_dataloader):
    current_data = data[0][0]

    current_data_np = current_data.numpy()
    np.save(f"data/evaluation_npy_step1-250/gt_{i:03d}.npy", current_data_np)

# Generate High Resolution Dataset

In [ ]:
import os
import numpy as np

def upsample_grid(grid, scale=2):
    """
    Upsample a 3D grid by integer scale, filling new cells with 0.
    """
    D, H, W = grid.shape
    new_D, new_H, new_W = D*scale, H*scale, W*scale
    up_grid = np.zeros((new_D, new_H, new_W), dtype=grid.dtype)
    up_grid[::scale, ::scale, ::scale] = grid
    return up_grid

def process_folder(input_dir, output_dir, scale=2):
    """
    Process all .npy files in input_dir and save upsampled grids to output_dir.
    """
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith(".npy"):
            file_path = os.path.join(input_dir, filename)
            grid = np.load(file_path)

            up_grid = upsample_grid(grid, scale=scale)

            save_path = os.path.join(output_dir, filename)
            np.save(save_path, up_grid)
            print(f"Processed {filename} -> {save_path}")

# 使用示例
input_dir = "/root/autodl-tmp/SCALED-Particle/data/couple_spout_3D"
output_dir = "/root/autodl-tmp/SCALED-Particle/data/high_res"
process_folder(input_dir, output_dir, scale=2)


In [ ]:
from src.scaled.dataset.particle_fluid_dataset import ParticleFluidDataset
from torch.utils.data import DataLoader
import numpy as np

val_dataset = ParticleFluidDataset(
        data_dir="data/high_res",
        skip_timestep=1,
        time_steps_list=[i for i in range(200, 250)],
    )

val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=1)

for i, data in enumerate(val_dataloader):
    current_data = data[0][0]

    current_data_np = current_data.numpy()
    np.save(f"data/high_res_npy/gt_{i:03d}.npy", current_data_np)

# Generate SFC

In [2]:
import torch
import numpy as np

def morton_sfc_3d(D, H, W):
    """
    生成任意长方体 (D,H,W) 的 Morton / Z-order SFC 通道
    输出 shape: (D,H,W)，归一化到 [0,1]
    """
    # 最大维度决定 bit 数
    n = max(D, H, W).bit_length()

    def part1by2(n):
        n &= 0x1FFFFF
        n = (n | (n << 32)) & 0x1F00000000FFFF
        n = (n | (n << 16)) & 0x1F0000FF0000FF
        n = (n | (n << 8)) & 0x100F00F00F00F00F
        n = (n | (n << 4)) & 0x10C30C30C30C30C3
        n = (n | (n << 2)) & 0x1249249249249249
        return n

    vol = np.zeros((D, H, W), dtype=np.float64)

    for z in range(D):
        for y in range(H):
            for x in range(W):
                # 映射到 [0, 2^n-1]
                xm = int(x * (2**n - 1) / (W - 1)) if W > 1 else 0
                ym = int(y * (2**n - 1) / (H - 1)) if H > 1 else 0
                zm = int(z * (2**n - 1) / (D - 1)) if D > 1 else 0

                idx = part1by2(xm) | (part1by2(ym) << 1) | (part1by2(zm) << 2)
                vol[z, y, x] = idx

    return torch.from_numpy(vol)

morton_sfc = morton_sfc_3d(256, 64, 64)


nomalized_sfc = (morton_sfc - morton_sfc.min()) / (morton_sfc.max() - morton_sfc.min())

torch.save(nomalized_sfc, "data/morton_sfc.pt")